In [4]:
import os
import sys
import configparser

configread=configparser.ConfigParser()
configread.read(r'../config.ini')
s_path=configread.get('spark_env','spark_env_file')
h_path=configread.get('hadoop_pth','hadoop_pth_file')
sp_bin=configread.get('spark_bin','spark_bin_file')
sp_py=configread.get('spark_py','spark_py_file')
sp_pysp=configread.get('spark_pysp','spark_pysp_file')
sp_pylib=configread.get('spark_pylib','spark_pylib_file')
sp_pylib_pysp=configread.get('spark_pylib_pysp','spark_pylib_pysp_file')
sp_pylib_py4j=configread.get('spark_pylib_py4j','spark_pylib_py4j_file')
h_bin=configread.get('hadoop_bin','hadoop_bin_file')
input_file=configread.get('intput_path','input_file')

spark_path = s_path
hadoop_path = h_path
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = hadoop_path
sys.path.append(sp_bin)
sys.path.append(sp_py)
sys.path.append(sp_pysp)
sys.path.append(sp_pylib)
sys.path.append(sp_pylib_pysp)
sys.path.append(sp_pylib_py4j)
sys.path.append(h_bin)

In [5]:
from pyspark import SparkConf, SparkContext

In [6]:
conf=SparkConf().setAppName("Transformation_Actions").setMaster('local[4]')

In [7]:
sc=SparkContext(conf=conf)

In [8]:
r=sc.textFile(input_file,3)

In [9]:
def show(iterator): yield list(iterator)

In [10]:
r1=r.map(lambda x:x.split(','))

In [11]:
h=r1.first()

In [12]:
r1.filter(lambda x:x!=h).mapPartitions(show).collect()

[[['V Kohli',
   '5426',
   '152',
   '4111',
   '35.69736842105263',
   '131.98735100948676'],
  ['SK Raina', '5386', '160', '3916', '33.6625', '137.53830439223697'],
  ['RG Sharma',
   '4902',
   '161',
   '3742',
   '30.4472049689441',
   '130.99946552645645']],
 [['DA Warner',
   '4717',
   '114',
   '3292',
   '41.37719298245614',
   '143.28675577156744'],
  ['S Dhawan',
   '4601',
   '137',
   '3665',
   '33.583941605839414',
   '125.53888130968622'],
  ['CH Gayle',
   '4525',
   '110',
   '2972',
   '41.13636363636363',
   '152.2543741588156']],
 [['MS Dhoni',
   '4450',
   '118',
   '3206',
   '37.71186440677966',
   '138.80224578914536'],
  ['RV Uthappa',
   '4420',
   '156',
   '3381',
   '28.333333333333332',
   '130.73055309080155'],
  ['AB de Villiers',
   '4414',
   '104',
   '2902',
   '42.44230769230769',
   '152.10199862164023']]]

In [13]:
rd=sc.parallelize([(1,2),(3,4),(5,6),(7,8)],3)

In [14]:
def func(iterator):
    for i in iterator:
        print(i)
    print('----')

In [15]:
rd.foreachPartition(func)

In [16]:
rd.getNumPartitions()

3

In [17]:
def ifunc(iterator):
    yield lambda x:x.split(',')

In [ ]:
rd.mapPartitions(ifunc).collect()

In [ ]:
sum([1,2,3,4])

In [ ]:
parallel = sc.parallelize([(1,2),(3,4),(5,6),(7,8)],3)

In [ ]:
def show(iterator): yield list(iterator)

In [ ]:
parallel.mapPartitions(show).collect()

In [ ]:
b='ssadda dfafda dada adada adada aDAD'.split(' ')

In [ ]:
b1=sc.parallelize(b,2)

In [ ]:
sd={"spark":100,"sqoop":200,"big":300,"simple":400}

In [ ]:
sb=sc.broadcast(sd)

In [ ]:
sb.value

In [ ]:
b1.map(lambda x:(x,sb.value.get(x,0))).sortBy(lambda x:x[1]).collect()